In [82]:
import numpy as np
import pandas as pd
from scipy import spatial, stats, cluster
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nilearn import plotting, input_data
from nilearn.datasets import fetch_atlas_schaefer_2018
import networkx as nx
import matplotlib.cm as mplcm
import matplotlib.colors as colors
import bct
import glob
import community as community_louvain
import matplotlib.cm as cm
from sklearn.metrics import adjusted_rand_score
import os


%matplotlib inline

# Styles
sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower


In [83]:
dfLabels = pd.read_csv("../resources/aal_extended.csv",header=None) 
dfLabels.columns = ['reindex','name','mark','region','side']
dfLabels["region_side"] = dfLabels["region"] + "_" + dfLabels["side"]

print(dfLabels.head())

labels = dfLabels['name'].values

   reindex               name  mark   region side region_side
0        1       Precentral_L  2001  Central    L   Central_L
1        2       Precentral_R  2002  Central    R   Central_R
2        3      Frontal_Sup_L  2101  Frontal    L   Frontal_L
3        4      Frontal_Sup_R  2102  Frontal    R   Frontal_R
4        5  Frontal_Sup_Orb_L  2111  Frontal    L   Frontal_L


In [84]:
sleep_states = ['N1','N2','N3']


In [86]:
tipo_red="W"

path = r'../resources/DataSujetos' # use your path
all_files = glob.glob(path + f"/{tipo_red}_suj*.csv")

dict_rand = {}

for filename_W in all_files:
    #print(filename_W)
    
    df = pd.read_csv(filename_W, header=None, index_col=None,)
    thresh_cmat = (df.values > .4) * 1.0
    label_data = pd.DataFrame({'number': np.arange(len(labels)) + 1, 'region_side': labels})
    GW = nx.from_numpy_matrix(thresh_cmat)
    GW = nx.relabel_nodes(GW, lambda x: label_data['number'].tolist()[x])
    
    modulos = community_louvain.best_partition(GW)
    values_obs = [v for n,v in modulos.items()]
    #print({filename_W: set(values_obs)})


    for sleep_state in sleep_states:
        filename_Nx = filename_W.replace("W_", sleep_state+"_")
        
        df = pd.read_csv(filename_Nx, header=None, index_col=None,)
        thresh_cmat = (df.values > .4) * 1.0
        label_data = pd.DataFrame({'number': np.arange(len(labels)) + 1, 'region_side': labels})
        GNx = nx.from_numpy_matrix(thresh_cmat)
        GNx = nx.relabel_nodes(GNx, lambda x: label_data['number'].tolist()[x])

        modulos = community_louvain.best_partition(GNx)
        values_per = [v for n,v in modulos.items()]
        #print({filename_W: set(values_obs)})

        #print({os.path.basename(filename_Nx):adjusted_rand_score(values_obs, values_per)})
        
        dict_rand[os.path.basename(filename_Nx)] = adjusted_rand_score(values_obs, values_per)
        
    #break
dict_rand

{'N1_suj1.csv': 0.10581230226607159,
 'N2_suj1.csv': 0.16943927431422753,
 'N3_suj1.csv': 0.13273263177252934,
 'N1_suj10.csv': -0.0021695335502866365,
 'N2_suj10.csv': 0.0051095365304946695,
 'N3_suj10.csv': 0.004051559257534492,
 'N1_suj11.csv': 0.386960094068921,
 'N2_suj11.csv': 0.3075810330175476,
 'N3_suj11.csv': 0.31684622581032873,
 'N1_suj12.csv': 0.3265577054007185,
 'N2_suj12.csv': 0.07319976251551745,
 'N3_suj12.csv': 0.10817142617222252,
 'N1_suj13.csv': 0.2586835187268155,
 'N2_suj13.csv': 0.38505034716000386,
 'N3_suj13.csv': 0.1701187720645781,
 'N1_suj14.csv': 0.09789595458154257,
 'N2_suj14.csv': 0.08297811540562967,
 'N3_suj14.csv': 0.16753529057433938,
 'N1_suj15.csv': 0.035839877604329905,
 'N2_suj15.csv': 0.020380447306726803,
 'N3_suj15.csv': 0.07849709910825166,
 'N1_suj16.csv': 0.38710160585397574,
 'N2_suj16.csv': 0.4126157331849158,
 'N3_suj16.csv': 0.5590529532204548,
 'N1_suj17.csv': 0.1585882515119183,
 'N2_suj17.csv': 0.13160886223123283,
 'N3_suj17.csv':

In [64]:
df = pd.read_csv(fr'../resources/DataSujetos/W_suj1.csv', header=None, index_col=None,)

#thresh_cmat = bct.threshold_absolute(df.values, thr=.8)
thresh_cmat = (df.values > .4) * 1.0

label_data = pd.DataFrame({'number': np.arange(len(labels)) + 1, 'region_side': labels})

GW = nx.from_numpy_matrix(thresh_cmat)
GW = nx.relabel_nodes(GW, lambda x: label_data['number'].tolist()[x])



In [65]:
df = pd.read_csv(fr'../resources/DataSujetos\N3_suj1.csv', header=None, index_col=None,)

# thresh_cmat = bct.threshold_absolute(df.values, thr=.4)
thresh_cmat = (df.values > .4) * 1.0


label_data = pd.DataFrame({'number': np.arange(len(labels)) + 1, 'region_side': labels})

GNx = nx.from_numpy_matrix(thresh_cmat)
GNx = nx.relabel_nodes(GNx, lambda x: label_data['number'].tolist()[x])



In [66]:
modulos = community_louvain.best_partition(GW)
values_obs = [v for n,v in modulos.items()]
set(values_obs)

{0, 1, 2, 3, 4}

In [60]:
modulos = community_louvain.best_partition(GNx)
values_per = [v for n,v in modulos.items()]
set(values_obs)

{0, 1, 2, 3, 4}

In [62]:
from sklearn.metrics import adjusted_rand_score

{'N3_suj1':adjusted_rand_score(values_obs, values_per)}

{'N3_suj1': 0.1189657970977296}

In [31]:
all_files

['../resources/DataSujetos\\W_suj1.csv',
 '../resources/DataSujetos\\W_suj10.csv',
 '../resources/DataSujetos\\W_suj11.csv',
 '../resources/DataSujetos\\W_suj12.csv',
 '../resources/DataSujetos\\W_suj13.csv',
 '../resources/DataSujetos\\W_suj14.csv',
 '../resources/DataSujetos\\W_suj15.csv',
 '../resources/DataSujetos\\W_suj16.csv',
 '../resources/DataSujetos\\W_suj17.csv',
 '../resources/DataSujetos\\W_suj18.csv',
 '../resources/DataSujetos\\W_suj2.csv',
 '../resources/DataSujetos\\W_suj3.csv',
 '../resources/DataSujetos\\W_suj4.csv',
 '../resources/DataSujetos\\W_suj5.csv',
 '../resources/DataSujetos\\W_suj6.csv',
 '../resources/DataSujetos\\W_suj7.csv',
 '../resources/DataSujetos\\W_suj8.csv',
 '../resources/DataSujetos\\W_suj9.csv']